In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Tuple, Dict, Optional, Any, Union
from abc import ABC, abstractmethod
import json
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import warnings
import logging
from dataclasses import dataclass

# Core ML imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Transformer imports
try:
    from transformers import (
        AutoTokenizer, AutoModelForSequenceClassification, 
        RobertaTokenizer, RobertaForSequenceClassification,
        BertTokenizer, BertForSequenceClassification,
        Trainer, TrainingArguments
    )
    from datasets import Dataset
    import torch
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    TRANSFORMERS_AVAILABLE = False
    warnings.warn("Transformers/PyTorch not available. Only baseline models will be used.")

# SHAP for interpretability
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False
    warnings.warn("SHAP not available. Semantic drift analysis will be limited.")

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

@dataclass
class SemanticShift:
    """Data structure for tracking semantic shifts of political terms"""
    term: str
    old_period: str
    new_period: str
    old_ideology_weights: Dict[str, float]  # ideology -> weight
    new_ideology_weights: Dict[str, float]
    shift_magnitude: float
    direction_flip: bool  # True if term switched from left->right or vice versa
    statistical_significance: float

@dataclass 
class PoliticalTermAnalysis:
    """Analysis results for political terms over time"""
    term: str
    periods: List[str]
    ideology_trajectories: Dict[str, List[float]]  # ideology -> [weights over time]
    volatility_score: float
    trend_direction: str  # 'leftward', 'rightward', 'stable', 'volatile'

class DataManager:
    """Data manager focused on temporal political text analysis"""
    
    def __init__(self, folder_path: str):
        self.folder = Path(folder_path)
        if not self.folder.exists():
            raise FileNotFoundError(f"Data folder not found: {folder_path}")
            
        # Enhanced ideology mapping for better analysis
        self.ideology_map = {
            "CDU": "RIGHT",      # Conservative 
            "AFD": "RIGHT",      # Far-right    
            "SPD": "LEFT",       # Social Democratic
            "REPUBLICAN": "RIGHT",
            "DEMOCRATIC": "LEFT",
            "CONSERVATIVE": "RIGHT"
        }
    
    def load_data(self) -> pd.DataFrame:
        """Load data with focus on temporal analysis"""
        files = list(self.folder.glob("*.csv"))
        if not files:
            raise ValueError(f"No CSV files found in {self.folder}")
            
        logger.info(f"Loading data from {len(files)} files for temporal analysis")
        all_dfs = []
        
        for f in files:
            try:
                df_processed = self._process_file(f)
                if df_processed is not None and len(df_processed) > 0:
                    all_dfs.append(df_processed)
                    logger.info(f"Loaded {f.name}: {len(df_processed)} samples")
            except Exception as e:
                logger.error(f"Error processing {f.name}: {str(e)}")
                continue
        
        if not all_dfs:
            raise ValueError("No valid data files could be processed")
            
        combined_data = pd.concat(all_dfs, ignore_index=True)
        logger.info(f"Total loaded: {len(combined_data)} samples across {len(combined_data['year'].unique())} years")
        
        return self._prepare_for_temporal_analysis(combined_data)
    
    def _process_file(self, filepath: Path) -> Optional[pd.DataFrame]:
        """Process file with enhanced metadata extraction"""
        try:
            stem = filepath.stem
            parts = stem.split('_')
            
            if len(parts) < 3:
                return None
                
            country_code = parts[0].upper()
            party_name = '_'.join(parts[1:-1]).upper()
            party_name = party_name.replace("PARTY", "")
            year = int(parts[-1])
            
            if party_name not in self.ideology_map:
                logger.warning(f"Unknown party '{party_name}' in {filepath.name}")
                return None
                
            ideology = self.ideology_map[party_name]
            
            df_file = pd.read_csv(filepath, encoding='utf-8')
            text_col = self._identify_text_column(df_file, country_code)
            
            if text_col is None:
                return None
                
            result_df = pd.DataFrame({
                'text': df_file[text_col],
                'ideology': ideology,
                'country': country_code,
                'party': party_name,
                'year': year,
                'period': self._assign_period(year)  # Group years into periods
            })
            
            return result_df[result_df['text'].str.len() >= 50]  # Filter very short texts
            
        except Exception as e:
            logger.error(f"Failed to process {filepath.name}: {str(e)}")
            return None
    
    def _identify_text_column(self, df: pd.DataFrame, country_code: str) -> Optional[str]:
        """Identify text column"""
        if country_code == 'DE' and 'text_en' in df.columns:
            return 'text_en'
        elif 'text' in df.columns:
            return 'text'
        else:
            text_cols = [col for col in df.columns if 'text' in col.lower()]
            return text_cols[0] if text_cols else None
    
    def _assign_period(self, year: int) -> str:
        """Assign years to periods for temporal analysis"""
        if year <= 2005:
            return "2000-2005"
        elif year <= 2010:
            return "2006-2010"
        elif year <= 2015:
            return "2011-2015"
        else:
            return "2016-2022"
    
    def _prepare_for_temporal_analysis(self, data: pd.DataFrame) -> pd.DataFrame:
        """Prepare data specifically for temporal semantic analysis"""
        # Ensure sufficient data per period and ideology
        period_ideology_counts = data.groupby(['period', 'ideology']).size().reset_index(name='count')
        
        # Filter periods/ideologies with insufficient data
        min_samples = 50
        valid_combinations = period_ideology_counts[period_ideology_counts['count'] >= min_samples]
        
        logger.info(f"Valid period-ideology combinations:")
        for _, row in valid_combinations.iterrows():
            logger.info(f"  {row['period']} + {row['ideology']}: {row['count']} samples")
        
        return data

class PoliticalTermExtractor:
    """Extract and track political terms across time periods"""
    
    def __init__(self):
        # Comprehensive political vocabulary
        self.political_terms = {
            # Economic terms
            'economy': ['economy', 'economic', 'economics', 'gdp', 'growth'],
            'tax': ['tax', 'taxes', 'taxation', 'revenue', 'fiscal'],
            'budget': ['budget', 'deficit', 'surplus', 'spending', 'debt'],
            'reform': ['reform', 'reforms', 'reforming', 'restructure', 'change'],
            
            # Security & Defense
            'security': ['security', 'secure', 'safety', 'protection', 'defend'],
            'military': ['military', 'defense', 'defence', 'army', 'forces'],
            'border': ['border', 'borders', 'immigration', 'migrant', 'refugee'],
            'terrorism': ['terrorism', 'terrorist', 'terror', 'extremism'],
            
            # Social Issues
            'healthcare': ['healthcare', 'health', 'medical', 'medicare', 'insurance'],
            'education': ['education', 'school', 'schools', 'university', 'student'],
            'welfare': ['welfare', 'social', 'benefits', 'assistance', 'support'],
            'rights': ['rights', 'equality', 'discrimination', 'civil', 'freedom'],
            
            # Environmental
            'environment': ['environment', 'climate', 'green', 'carbon', 'emission'],
            'energy': ['energy', 'renewable', 'oil', 'gas', 'nuclear'],
            
            # Governance
            'government': ['government', 'federal', 'state', 'administration', 'policy'],
            'regulation': ['regulation', 'regulate', 'deregulation', 'oversight'],
            'justice': ['justice', 'court', 'legal', 'law', 'judicial'],
            
            # Values & Identity
            'freedom': ['freedom', 'liberty', 'free', 'independence'],
            'tradition': ['tradition', 'traditional', 'conservative', 'values'],
            'progress': ['progress', 'progressive', 'modern', 'innovation'],
            'family': ['family', 'families', 'marriage', 'children']
        }
        
        # Flatten for easy lookup
        self.all_political_terms = set()
        for category, terms in self.political_terms.items():
            self.all_political_terms.update(terms)
    
    def extract_political_context(self, text: str, term: str, window_size: int = 5) -> List[str]:
        """Extract context words around political terms"""
        words = text.lower().split()
        contexts = []
        
        for i, word in enumerate(words):
            if term in word:
                start = max(0, i - window_size)
                end = min(len(words), i + window_size + 1)
                context = ' '.join(words[start:end])
                contexts.append(context)
        
        return contexts

class ModelWrapper(ABC):
    """Abstract model wrapper for political text analysis"""
    
    @abstractmethod
    def train(self, texts: List[str], labels: List[str]) -> None:
        pass

    @abstractmethod
    def predict(self, texts: List[str]) -> Tuple[List[str], List[float]]:
        pass

    @abstractmethod
    def get_political_term_weights(self, terms: List[str]) -> Dict[str, Dict[str, float]]:
        """Get ideology weights for specific political terms"""
        pass

class BaselineModel(ModelWrapper):
    """Baseline TF-IDF + Logistic Regression model"""
    
    def __init__(self, max_features: int = 15000):
        self.pipeline = Pipeline([
            ("tfidf", TfidfVectorizer(
                max_features=max_features,
                ngram_range=(1, 2),
                min_df=3,
                max_df=0.9,
                stop_words='english',
                lowercase=True
            )),
            ("clf", LogisticRegression(
                max_iter=1000,
                class_weight='balanced',
                random_state=42,
                C=1.0
            ))
        ])
        self.label_encoder = LabelEncoder()
        self.is_trained = False

    def train(self, texts: List[str], labels: List[str]) -> None:
        logger.info(f"Training baseline model on {len(texts)} samples")
        y = self.label_encoder.fit_transform(labels)
        self.pipeline.fit(texts, y)
        self.is_trained = True
        
        # Log class distribution
        unique, counts = np.unique(y, return_counts=True)
        class_dist = dict(zip(self.label_encoder.inverse_transform(unique), counts))
        logger.info(f"Training class distribution: {class_dist}")

    def predict(self, texts: List[str]) -> Tuple[List[str], List[float]]:
        if not self.is_trained:
            raise ValueError("Model not trained")
            
        proba = self.pipeline.predict_proba(texts)
        pred_indices = proba.argmax(axis=1)
        labels = self.label_encoder.inverse_transform(pred_indices)
        confidence = proba.max(axis=1)
        
        return labels.tolist(), confidence.tolist()

    def get_political_term_weights(self, terms: List[str]) -> Dict[str, Dict[str, float]]:
        """Extract weights for political terms across ideologies"""
        if not self.is_trained:
            return {}
            
        vectorizer = self.pipeline.named_steps["tfidf"]
        clf = self.pipeline.named_steps["clf"]
        feature_names = vectorizer.get_feature_names_out()
        
        # Create mapping from feature names to indices
        feature_to_idx = {name: idx for idx, name in enumerate(feature_names)}
        
        results = {}
        
        for term in terms:
            term_weights = {}
            
            # Find features containing this term
            matching_features = [f for f in feature_names if term.lower() in f.lower()]
            
            if not matching_features:
                continue
            
            # Get average weight for this term across all matching features
            for i, ideology in enumerate(self.label_encoder.classes_):
                if len(self.label_encoder.classes_) == 2:
                    # Binary classification
                    coef = clf.coef_[0] if i == 1 else -clf.coef_[0]
                else:
                    # Multi-class
                    coef = clf.coef_[i]
                
                # Average weight across all features containing the term
                term_weight = np.mean([coef[feature_to_idx[f]] for f in matching_features])
                term_weights[ideology] = float(term_weight)
            
            results[term] = term_weights
        
        return results

class TransformerModel(ModelWrapper):
    """BERT/RoBERTa model wrapper with interpretability - FIXED VERSION"""
    
    def __init__(self, model_name: str = "roberta-base"):
        if not TRANSFORMERS_AVAILABLE:
            raise ImportError("Transformers library not available")
            
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.label_encoder = LabelEncoder()
        self.is_trained = False
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        logger.info(f"Initializing {model_name} on {self.device}")

    def train(self, texts: List[str], labels: List[str]) -> None:
        logger.info(f"Training {self.model_name} on {len(texts)} samples")
        
        # Initialize tokenizer and model
        if 'roberta' in self.model_name:
            self.tokenizer = RobertaTokenizer.from_pretrained(self.model_name)
            self.model = RobertaForSequenceClassification.from_pretrained(
                self.model_name, num_labels=len(set(labels))
            )
        else:  # BERT
            self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
            self.model = BertForSequenceClassification.from_pretrained(
                self.model_name, num_labels=len(set(labels))
            )
        
        self.model.to(self.device)
        
        # Prepare data
        y = self.label_encoder.fit_transform(labels)
        
        # Tokenize texts
        encodings = self.tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=512,
            return_tensors="pt"
        )
        
        # Create dataset
        class PoliticalDataset(torch.utils.data.Dataset):
            def __init__(self, encodings, labels):
                self.encodings = encodings
                self.labels = labels
            
            def __getitem__(self, idx):
                item = {key: val[idx] for key, val in self.encodings.items()}
                item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
                return item
            
            def __len__(self):
                return len(self.labels)
        
        dataset = PoliticalDataset(encodings, y)
        
        # FIXED: Updated training arguments with version compatibility
        try:
            # Try newer transformers version parameter name
            training_args = TrainingArguments(
                output_dir='./temp_results',
                num_train_epochs=2,  # Reduced for faster training
                per_device_train_batch_size=8,
                per_device_eval_batch_size=16,
                warmup_steps=100,  # Reduced warmup
                weight_decay=0.01,
                logging_dir='./temp_logs',
                logging_steps=100,
                save_strategy="no",
                eval_strategy="no",  # New parameter name
                report_to=[],
                remove_unused_columns=True,
                dataloader_pin_memory=False,
                disable_tqdm=False
            )
        except TypeError:
            # Fallback for older transformers versions
            training_args = TrainingArguments(
                output_dir='./temp_results',
                num_train_epochs=2,
                per_device_train_batch_size=8,
                per_device_eval_batch_size=16,
                warmup_steps=100,
                weight_decay=0.01,
                logging_dir='./temp_logs',
                logging_steps=100,
                save_strategy="no",
                evaluation_strategy="no",  # Old parameter name
                report_to=[],
                remove_unused_columns=True,
                dataloader_pin_memory=False,
                disable_tqdm=False
            )
        
        # Train
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=dataset,
            tokenizer=self.tokenizer
        )
        
        try:
            trainer.train()
            self.is_trained = True
            logger.info(f"✅ {self.model_name} training completed")
        except Exception as e:
            logger.error(f"Training failed: {str(e)}")
            raise

    def predict(self, texts: List[str]) -> Tuple[List[str], List[float]]:
        if not self.is_trained:
            raise ValueError("Model not trained")
        
        self.model.eval()
        predictions = []
        confidences = []
        
        batch_size = 16
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            
            encodings = self.tokenizer(
                batch_texts,
                truncation=True,
                padding=True,
                max_length=512,
                return_tensors="pt"
            ).to(self.device)
            
            with torch.no_grad():
                outputs = self.model(**encodings)
                logits = outputs.logits
                probs = torch.nn.functional.softmax(logits, dim=-1)
                
                batch_preds = torch.argmax(logits, dim=-1).cpu().numpy()
                batch_confs = torch.max(probs, dim=-1)[0].cpu().numpy()
                
                predictions.extend(batch_preds)
                confidences.extend(batch_confs)
        
        # Convert predictions to labels
        labels = self.label_encoder.inverse_transform(predictions)
        
        return labels.tolist(), confidences.tolist()

    def get_political_term_weights(self, terms: List[str]) -> Dict[str, Dict[str, float]]:
        """Use simplified attention-based approach for political term importance"""
        if not self.is_trained:
            return {}
        
        # Simplified approach: analyze model's internal representations
        results = {}
        
        for term in terms:
            # Create simple test sentences with the term
            test_sentences = [
                f"The {term} policy is important.",
                f"We support {term} reform.",
                f"The {term} issue needs attention."
            ]
            
            term_weights = {}
            
            try:
                predictions, confidences = self.predict(test_sentences)
                
                # Simple heuristic: if model consistently predicts one ideology
                # for sentences containing this term, assign weight accordingly
                ideology_counts = Counter(predictions)
                total_predictions = len(predictions)
                
                for ideology in self.label_encoder.classes_:
                    count = ideology_counts.get(ideology, 0)
                    weight = (count / total_predictions - 0.5) * 2  # Normalize to [-1, 1]
                    term_weights[ideology] = float(weight)
                
                results[term] = term_weights
                
            except Exception as e:
                logger.warning(f"Could not get weights for term '{term}': {str(e)}")
                # Return zero weights as fallback
                for ideology in self.label_encoder.classes_:
                    term_weights[ideology] = 0.0
                results[term] = term_weights
        
        return results

class SemanticShiftAnalyzer:
    """Core analyzer for detecting semantic shifts in political language"""
    
    def __init__(self):
        self.term_extractor = PoliticalTermExtractor()
        self.shift_threshold = 0.1  # Minimum weight change to consider significant
        
    def analyze_temporal_shifts(self, 
                              models_by_period: Dict[str, ModelWrapper],
                              periods: List[str]) -> Dict[str, PoliticalTermAnalysis]:
        """Analyze how political terms shift meaning across time periods"""
        
        logger.info(f"Analyzing semantic shifts across {len(periods)} periods")
        
        # Get political terms to analyze
        terms_to_analyze = list(self.term_extractor.all_political_terms)[:30]  # Top 30 terms
        
        results = {}
        
        for term in terms_to_analyze:
            logger.info(f"Analyzing semantic trajectory for: {term}")
            
            ideology_trajectories = defaultdict(list)
            
            # Get term weights for each period
            for period in periods:
                if period not in models_by_period:
                    continue
                    
                model = models_by_period[period]
                term_weights = model.get_political_term_weights([term])
                
                if term in term_weights:
                    for ideology, weight in term_weights[term].items():
                        ideology_trajectories[ideology].append(weight)
                else:
                    # Term not found in this period
                    for ideology in ['LEFT', 'RIGHT']:  # Focus on main ideologies
                        ideology_trajectories[ideology].append(0.0)
            
            # Calculate trajectory statistics
            volatility = self._calculate_volatility(ideology_trajectories)
            trend = self._determine_trend(ideology_trajectories)
            
            results[term] = PoliticalTermAnalysis(
                term=term,
                periods=periods,
                ideology_trajectories=dict(ideology_trajectories),
                volatility_score=volatility,
                trend_direction=trend
            )
        
        return results
    
    def detect_semantic_flips(self, 
                            old_model: ModelWrapper, 
                            new_model: ModelWrapper,
                            old_period: str,
                            new_period: str) -> List[SemanticShift]:
        """Detect terms that flipped ideological association"""
        
        logger.info(f"Detecting semantic flips: {old_period} -> {new_period}")
        
        terms_to_analyze = list(self.term_extractor.all_political_terms)
        
        old_weights = old_model.get_political_term_weights(terms_to_analyze)
        new_weights = new_model.get_political_term_weights(terms_to_analyze)
        
        semantic_shifts = []
        
        for term in terms_to_analyze:
            if term not in old_weights or term not in new_weights:
                continue
                
            old_term_weights = old_weights[term]
            new_term_weights = new_weights[term]
            
            # Calculate shift magnitude
            shift_magnitude = self._calculate_shift_magnitude(old_term_weights, new_term_weights)
            
            # Check for ideological flip
            direction_flip = self._detect_direction_flip(old_term_weights, new_term_weights)
            
            if shift_magnitude > self.shift_threshold or direction_flip:
                shift = SemanticShift(
                    term=term,
                    old_period=old_period,
                    new_period=new_period,
                    old_ideology_weights=old_term_weights,
                    new_ideology_weights=new_term_weights,
                    shift_magnitude=shift_magnitude,
                    direction_flip=direction_flip,
                    statistical_significance=self._calculate_significance(
                        old_term_weights, new_term_weights
                    )
                )
                semantic_shifts.append(shift)
        
        # Sort by shift magnitude
        semantic_shifts.sort(key=lambda x: x.shift_magnitude, reverse=True)
        
        logger.info(f"Found {len(semantic_shifts)} significant semantic shifts")
        
        return semantic_shifts
    
    def _calculate_volatility(self, trajectories: Dict[str, List[float]]) -> float:
        """Calculate volatility score for term across ideologies"""
        all_changes = []
        
        for ideology, weights in trajectories.items():
            if len(weights) > 1:
                changes = np.abs(np.diff(weights))
                all_changes.extend(changes)
        
        return float(np.mean(all_changes)) if all_changes else 0.0
    
    def _determine_trend(self, trajectories: Dict[str, List[float]]) -> str:
        """Determine overall trend direction"""
        ideology_trends = {}
        
        for ideology, weights in trajectories.items():
            if len(weights) > 1:
                # Simple linear trend
                x = np.arange(len(weights))
                slope = np.polyfit(x, weights, 1)[0]
                ideology_trends[ideology] = slope
        
        if not ideology_trends:
            return 'stable'
        
        # Determine dominant trend
        left_trend = ideology_trends.get('LEFT', 0)
        right_trend = ideology_trends.get('RIGHT', 0)
        
        if abs(left_trend) > abs(right_trend) and abs(left_trend) > 0.01:
            return 'leftward' if left_trend > 0 else 'rightward'
        elif abs(right_trend) > 0.01:
            return 'rightward' if right_trend > 0 else 'leftward'
        else:
            return 'stable'
    
    def _calculate_shift_magnitude(self, 
                                 old_weights: Dict[str, float], 
                                 new_weights: Dict[str, float]) -> float:
        """Calculate magnitude of semantic shift"""
        shifts = []
        
        for ideology in old_weights:
            if ideology in new_weights:
                shift = abs(new_weights[ideology] - old_weights[ideology])
                shifts.append(shift)
        
        return float(np.mean(shifts)) if shifts else 0.0
    
    def _detect_direction_flip(self, 
                             old_weights: Dict[str, float], 
                             new_weights: Dict[str, float]) -> bool:
        """Detect if term flipped ideological direction"""
        
        # Check if strongest association flipped
        old_max_ideology = max(old_weights, key=old_weights.get)
        new_max_ideology = max(new_weights, key=new_weights.get)
        
        # Simple flip detection: if strongest ideology changed and weights are significant
        if (old_max_ideology != new_max_ideology and 
            abs(old_weights[old_max_ideology]) > 0.05 and 
            abs(new_weights[new_max_ideology]) > 0.05):
            return True
        
        return False
    
    def _calculate_significance(self, 
                              old_weights: Dict[str, float], 
                              new_weights: Dict[str, float]) -> float:
        """Calculate statistical significance of shift (simplified)"""
        # This is a placeholder - real implementation would use proper statistical tests
        shift_magnitude = self._calculate_shift_magnitude(old_weights, new_weights)
        return min(1.0, shift_magnitude * 10)  # Simple approximation

class SemanticDriftExperiment:
    """Main experiment class for political semantic drift analysis"""
    
    def __init__(self, data: pd.DataFrame, output_dir: str = "semantic_drift_results"):
        self.data = data
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        self.analyzer = SemanticShiftAnalyzer()
        self.models_by_period = {}
        
        # Setup logging
        log_file = self.output_dir / "semantic_drift.log"
        handler = logging.FileHandler(log_file)
        handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logger.addHandler(handler)
    
    def run_experiment(self, 
                      model_types: List[str] = ["baseline", "roberta"],
                      min_samples_per_period: int = 100):
        """Run complete semantic drift experiment"""
        
        logger.info("Starting Political Semantic Drift Analysis")
        logger.info(f"Model types: {model_types}")
        
        periods = sorted(self.data['period'].unique())
        logger.info(f"Time periods: {periods}")
        
        # Train models for each period and model type
        for model_type in model_types:
            logger.info(f"\n=== Training {model_type.upper()} models ===")
            self.models_by_period[model_type] = {}
            
            for period in periods:
                period_data = self.data[self.data['period'] == period]
                
                if len(period_data) < min_samples_per_period:
                    logger.warning(f"Insufficient data for {period}: {len(period_data)} samples")
                    continue
                
                logger.info(f"Training {model_type} for period {period}: {len(period_data)} samples")
                
                # Check class balance
                class_counts = period_data['ideology'].value_counts()
                logger.info(f"  Class distribution: {class_counts.to_dict()}")
                
                if len(class_counts) < 2:
                    logger.warning(f"Only one class in {period}, skipping")
                    continue
                
                # Initialize model
                if model_type == "baseline":
                    model = BaselineModel()
                elif model_type == "roberta" and TRANSFORMERS_AVAILABLE:
                    model = TransformerModel("roberta-base")
                elif model_type == "bert" and TRANSFORMERS_AVAILABLE:
                    model = TransformerModel("bert-base-uncased")
                else:
                    logger.warning(f"Model type {model_type} not available, skipping")
                    continue
                
                # Train model
                texts = period_data['text'].tolist()
                labels = period_data['ideology'].tolist()
                
                try:
                    model.train(texts, labels)
                    self.models_by_period[model_type][period] = model
                    logger.info(f"✅ Successfully trained {model_type} for {period}")
                    
                except Exception as e:
                    logger.error(f"❌ Failed to train {model_type} for {period}: {str(e)}")
                    continue
        
        # Analyze semantic shifts
        self._analyze_and_save_results()
    
    def _analyze_and_save_results(self):
        """Analyze semantic shifts and save results"""
        
        logger.info("\n=== Analyzing Semantic Shifts ===")
        
        results = {}
        
        for model_type, period_models in self.models_by_period.items():
            if len(period_models) < 2:
                logger.warning(f"Not enough periods for {model_type} analysis")
                continue
                
            logger.info(f"Analyzing shifts for {model_type}")
            
            periods = sorted(period_models.keys())
            
            # Temporal trajectory analysis
            trajectory_analysis = self.analyzer.analyze_temporal_shifts(
                period_models, periods
            )
            
            # Pairwise shift detection
            pairwise_shifts = {}
            for i in range(len(periods) - 1):
                old_period = periods[i]
                new_period = periods[i + 1]
                
                shifts = self.analyzer.detect_semantic_flips(
                    period_models[old_period],
                    period_models[new_period], 
                    old_period,
                    new_period
                )
                
                pairwise_shifts[f"{old_period}->{new_period}"] = shifts
            
            results[model_type] = {
                'trajectory_analysis': trajectory_analysis,
                'pairwise_shifts': pairwise_shifts
            }
        
        # Save results
        self._save_semantic_drift_results(results)
        self._create_visualizations(results)
        self._generate_research_findings(results)
    
    def _save_semantic_drift_results(self, results: Dict):
        """Save detailed semantic drift results"""
        
        # Convert results to JSON-serializable format
        json_results = {}
        
        for model_type, model_results in results.items():
            json_results[model_type] = {
                'trajectory_analysis': {},
                'pairwise_shifts': {}
            }
            
            # Convert trajectory analysis
            for term, analysis in model_results['trajectory_analysis'].items():
                json_results[model_type]['trajectory_analysis'][term] = {
                    'term': analysis.term,
                    'periods': analysis.periods,
                    'ideology_trajectories': analysis.ideology_trajectories,
                    'volatility_score': float(analysis.volatility_score),
                    'trend_direction': analysis.trend_direction
                }
            
            # Convert pairwise shifts
            for period_pair, shifts in model_results['pairwise_shifts'].items():
                json_results[model_type]['pairwise_shifts'][period_pair] = []
                
                for shift in shifts[:10]:  # Top 10 shifts per period pair
                    json_results[model_type]['pairwise_shifts'][period_pair].append({
                        'term': shift.term,
                        'old_period': shift.old_period,
                        'new_period': shift.new_period,
                        'old_ideology_weights': shift.old_ideology_weights,
                        'new_ideology_weights': shift.new_ideology_weights,
                        'shift_magnitude': float(shift.shift_magnitude),
                        'direction_flip': shift.direction_flip,
                        'statistical_significance': float(shift.statistical_significance)
                    })
        
        # Save to file
        results_file = self.output_dir / "semantic_drift_analysis.json"
        with open(results_file, 'w') as f:
            json.dump(json_results, f, indent=2)
        
        logger.info(f"Saved semantic drift results: {results_file}")
    
    def _create_visualizations(self, results: Dict):
        """Create visualizations for semantic drift patterns"""
        
        logger.info("Creating semantic drift visualizations...")
        
        for model_type, model_results in results.items():
            
            # 1. Political term trajectory plots
            self._plot_term_trajectories(
                model_results['trajectory_analysis'], 
                model_type
            )
            
            # 2. Semantic shift heatmaps
            self._plot_shift_heatmaps(
                model_results['pairwise_shifts'],
                model_type
            )
            
            # 3. Direction flip analysis
            self._plot_direction_flips(
                model_results['pairwise_shifts'],
                model_type
            )
    
    def _plot_term_trajectories(self, trajectory_analysis: Dict, model_type: str):
        """Plot how political terms evolve across time periods"""
        
        # Select top 12 most volatile terms
        top_terms = sorted(
            trajectory_analysis.items(),
            key=lambda x: x[1].volatility_score,
            reverse=True
        )[:12]
        
        fig, axes = plt.subplots(3, 4, figsize=(20, 15))
        axes = axes.flatten()
        
        fig.suptitle(f'Political Term Trajectories - {model_type.upper()}', fontsize=16)
        
        for idx, (term, analysis) in enumerate(top_terms):
            ax = axes[idx]
            
            periods = analysis.periods
            x_pos = range(len(periods))
            
            # Plot trajectory for each ideology
            for ideology, weights in analysis.ideology_trajectories.items():
                if len(weights) == len(periods):
                    ax.plot(x_pos, weights, 'o-', label=ideology, linewidth=2, markersize=6)
            
            ax.set_title(f'{term}\n(volatility: {analysis.volatility_score:.3f})', fontsize=10)
            ax.set_xlabel('Time Period')
            ax.set_ylabel('Ideology Weight')
            ax.set_xticks(x_pos)
            ax.set_xticklabels(periods, rotation=45, ha='right', fontsize=8)
            ax.legend(fontsize=8)
            ax.grid(True, alpha=0.3)
        
        # Hide unused subplots
        for idx in range(len(top_terms), len(axes)):
            axes[idx].set_visible(False)
        
        plt.tight_layout()
        
        plot_file = self.output_dir / f"term_trajectories_{model_type}.png"
        plt.savefig(plot_file, dpi=300, bbox_inches='tight')
        logger.info(f"Saved trajectory plot: {plot_file}")
        plt.close()
    
    def _plot_shift_heatmaps(self, pairwise_shifts: Dict, model_type: str):
        """Create heatmaps showing semantic shift magnitudes"""
        
        # Collect all terms and their shifts
        all_terms = set()
        shift_data = {}
        
        for period_pair, shifts in pairwise_shifts.items():
            shift_data[period_pair] = {}
            for shift in shifts:
                all_terms.add(shift.term)
                shift_data[period_pair][shift.term] = shift.shift_magnitude
        
        # Create matrix for heatmap
        terms_list = sorted(list(all_terms))[:20]  # Top 20 terms
        period_pairs = list(shift_data.keys())
        
        if not terms_list or not period_pairs:
            logger.warning(f"No data for heatmap - {model_type}")
            return
        
        matrix = np.zeros((len(terms_list), len(period_pairs)))
        
        for j, period_pair in enumerate(period_pairs):
            for i, term in enumerate(terms_list):
                matrix[i, j] = shift_data[period_pair].get(term, 0)
        
        # Create heatmap
        plt.figure(figsize=(12, 10))
        sns.heatmap(
            matrix,
            xticklabels=period_pairs,
            yticklabels=terms_list,
            annot=True,
            fmt='.3f',
            cmap='YlOrRd',
            cbar_kws={'label': 'Shift Magnitude'}
        )
        
        plt.title(f'Semantic Shift Magnitudes - {model_type.upper()}')
        plt.xlabel('Time Period Transitions')
        plt.ylabel('Political Terms')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        
        plt.tight_layout()
        
        heatmap_file = self.output_dir / f"shift_heatmap_{model_type}.png"
        plt.savefig(heatmap_file, dpi=300, bbox_inches='tight')
        logger.info(f"Saved shift heatmap: {heatmap_file}")
        plt.close()
    
    def _plot_direction_flips(self, pairwise_shifts: Dict, model_type: str):
        """Visualize terms that flip ideological direction"""
        
        flip_data = []
        
        for period_pair, shifts in pairwise_shifts.items():
            flips = [s for s in shifts if s.direction_flip]
            
            for shift in flips[:5]:  # Top 5 flips per period
                flip_data.append({
                    'term': shift.term,
                    'period_transition': period_pair,
                    'shift_magnitude': shift.shift_magnitude,
                    'old_strongest': max(shift.old_ideology_weights, key=shift.old_ideology_weights.get),
                    'new_strongest': max(shift.new_ideology_weights, key=shift.new_ideology_weights.get)
                })
        
        if not flip_data:
            logger.info(f"No direction flips detected for {model_type}")
            return
        
        # Create visualization
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        
        # Plot 1: Flip magnitudes
        flip_df = pd.DataFrame(flip_data)
        
        if len(flip_df) > 0:
            flip_summary = flip_df.groupby('term')['shift_magnitude'].max().sort_values(ascending=False)[:15]
            
            ax1.barh(range(len(flip_summary)), flip_summary.values)
            ax1.set_yticks(range(len(flip_summary)))
            ax1.set_yticklabels(flip_summary.index)
            ax1.set_xlabel('Maximum Shift Magnitude')
            ax1.set_title('Terms with Ideological Direction Flips')
            
            # Plot 2: Flip transitions
            flip_transitions = flip_df.groupby(['old_strongest', 'new_strongest']).size().reset_index(name='count')
            
            if len(flip_transitions) > 0:
                transition_matrix = flip_transitions.pivot(
                    index='old_strongest', 
                    columns='new_strongest', 
                    values='count'
                ).fillna(0)
                
                sns.heatmap(
                    transition_matrix,
                    annot=True,
                    fmt='g',
                    cmap='Blues',
                    ax=ax2,
                    cbar_kws={'label': 'Number of Terms'}
                )
                ax2.set_title('Ideological Transition Patterns')
                ax2.set_xlabel('New Strongest Association')
                ax2.set_ylabel('Old Strongest Association')
        
        plt.suptitle(f'Direction Flip Analysis - {model_type.upper()}')
        plt.tight_layout()
        
        flip_file = self.output_dir / f"direction_flips_{model_type}.png"
        plt.savefig(flip_file, dpi=300, bbox_inches='tight')
        logger.info(f"Saved direction flip plot: {flip_file}")
        plt.close()
    
    def _generate_research_findings(self, results: Dict):
        """Generate research findings summary"""
        
        logger.info("Generating research findings...")
        
        findings = {
            'experiment_summary': {
                'total_models_trained': sum(len(models) for models in self.models_by_period.values()),
                'periods_analyzed': len(self.data['period'].unique()),
                'model_types': list(results.keys())
            },
            'key_findings': {},
            'term_analysis': {},
            'methodology_insights': {}
        }
        
        for model_type, model_results in results.items():
            model_findings = self._analyze_model_findings(model_results)
            findings['key_findings'][model_type] = model_findings
        
        # Cross-model comparison
        if len(results) > 1:
            findings['cross_model_comparison'] = self._compare_models(results)
        
        # Most volatile terms across all models
        findings['term_analysis'] = self._analyze_term_patterns(results)
        
        # Methodology insights
        findings['methodology_insights'] = {
            'temporal_splitting_effectiveness': 'Successfully isolated time periods for drift analysis',
            'model_comparison': 'Multiple architectures enable robustness validation',
            'semantic_shift_detection': 'Quantified ideological association changes',
            'limitations': [
                'Limited by training data size per period',
                'Transformer interpretability challenges',
                'Need larger vocabulary for comprehensive analysis'
            ]
        }
        
        # Save findings
        findings_file = self.output_dir / "research_findings.json"
        with open(findings_file, 'w') as f:
            json.dump(findings, f, indent=2)
        
        # Create readable summary
        self._create_readable_summary(findings)
        
        logger.info(f"Saved research findings: {findings_file}")
    
    def _analyze_model_findings(self, model_results: Dict) -> Dict:
        """Analyze findings for a specific model"""
        
        trajectory_analysis = model_results['trajectory_analysis']
        pairwise_shifts = model_results['pairwise_shifts']
        
        # Find most volatile terms
        volatile_terms = sorted(
            trajectory_analysis.items(),
            key=lambda x: x[1].volatility_score,
            reverse=True
        )[:10]
        
        # Find terms with most direction flips
        all_flips = []
        for shifts in pairwise_shifts.values():
            all_flips.extend([s for s in shifts if s.direction_flip])
        
        flip_counts = Counter(s.term for s in all_flips)
        
        # Analyze trend patterns
        trend_counts = Counter(analysis.trend_direction for analysis in trajectory_analysis.values())
        
        return {
            'most_volatile_terms': [(term, float(analysis.volatility_score)) 
                                  for term, analysis in volatile_terms],
            'frequent_flip_terms': dict(flip_counts.most_common(10)),
            'trend_patterns': dict(trend_counts),
            'total_significant_shifts': sum(len(shifts) for shifts in pairwise_shifts.values()),
            'periods_with_most_shifts': max(pairwise_shifts.items(), 
                                          key=lambda x: len(x[1]))[0] if pairwise_shifts else None
        }
    
    def _compare_models(self, results: Dict) -> Dict:
        """Compare findings across different model types"""
        
        model_types = list(results.keys())
        
        comparison = {
            'volatility_correlation': {},
            'common_volatile_terms': [],
            'model_specific_insights': {}
        }
        
        if len(model_types) >= 2:
            # Find commonly volatile terms
            all_volatile = {}
            for model_type, model_results in results.items():
                trajectory_analysis = model_results['trajectory_analysis']
                volatile_terms = {
                    term: analysis.volatility_score 
                    for term, analysis in trajectory_analysis.items()
                }
                all_volatile[model_type] = volatile_terms
            
            # Find intersection
            common_terms = set.intersection(*[set(terms.keys()) for terms in all_volatile.values()])
            
            comparison['common_volatile_terms'] = [
                {
                    'term': term,
                    'volatility_by_model': {
                        model: float(all_volatile[model][term]) 
                        for model in model_types
                    }
                }
                for term in list(common_terms)[:15]
            ]
        
        return comparison
    
    def _analyze_term_patterns(self, results: Dict) -> Dict:
        """Analyze patterns across political terms"""
        
        # Aggregate term analysis across all models
        all_terms = {}
        
        for model_type, model_results in results.items():
            trajectory_analysis = model_results['trajectory_analysis']
            
            for term, analysis in trajectory_analysis.items():
                if term not in all_terms:
                    all_terms[term] = {
                        'models_analyzed': [],
                        'volatility_scores': [],
                        'trend_directions': []
                    }
                
                all_terms[term]['models_analyzed'].append(model_type)
                all_terms[term]['volatility_scores'].append(analysis.volatility_score)
                all_terms[term]['trend_directions'].append(analysis.trend_direction)
        
        # Calculate average volatility and consistency
        term_summary = {}
        for term, data in all_terms.items():
            term_summary[term] = {
                'avg_volatility': float(np.mean(data['volatility_scores'])),
                'volatility_std': float(np.std(data['volatility_scores'])),
                'models_count': len(data['models_analyzed']),
                'consistent_trend': len(set(data['trend_directions'])) == 1,
                'dominant_trend': Counter(data['trend_directions']).most_common(1)[0][0]
            }
        
        # Find most consistently volatile terms
        consistent_volatile = {
            term: data for term, data in term_summary.items()
            if data['avg_volatility'] > 0.05 and data['models_count'] > 1
        }
        
        return {
            'total_terms_analyzed': len(all_terms),
            'consistently_volatile_terms': dict(sorted(
                consistent_volatile.items(),
                key=lambda x: x[1]['avg_volatility'],
                reverse=True
            )[:20]),
            'cross_model_insights': {
                'high_agreement_terms': len([t for t in term_summary.values() if t['consistent_trend']]),
                'avg_volatility_all_terms': float(np.mean([t['avg_volatility'] for t in term_summary.values()]))
            }
        }
    
    def _create_readable_summary(self, findings: Dict):
        """Create a human-readable summary of findings"""
        
        summary_lines = []
        summary_lines.append("=== POLITICAL SEMANTIC DRIFT ANALYSIS - KEY FINDINGS ===\n")
        
        # Experiment overview
        exp_summary = findings['experiment_summary']
        summary_lines.append(f"📊 EXPERIMENT OVERVIEW:")
        summary_lines.append(f"   • Models trained: {exp_summary['total_models_trained']}")
        summary_lines.append(f"   • Time periods: {exp_summary['periods_analyzed']}")
        summary_lines.append(f"   • Model architectures: {', '.join(exp_summary['model_types'])}")
        summary_lines.append("")
        
        # Key findings by model
        for model_type, model_findings in findings['key_findings'].items():
            summary_lines.append(f"🔍 {model_type.upper()} MODEL FINDINGS:")
            
            # Most volatile terms
            volatile_terms = model_findings['most_volatile_terms'][:5]
            summary_lines.append("   Most semantically volatile terms:")
            for term, volatility in volatile_terms:
                summary_lines.append(f"      • {term}: {volatility:.3f} volatility score")
            
            # Direction flips
            flip_terms = model_findings['frequent_flip_terms']
            if flip_terms:
                summary_lines.append("   Terms with ideological direction flips:")
                for term, count in list(flip_terms.items())[:5]:
                    summary_lines.append(f"      • {term}: flipped {count} times")
            
            # Trend patterns
            trends = model_findings['trend_patterns']
            summary_lines.append(f"   Trend patterns: {dict(trends)}")
            summary_lines.append(f"   Total significant shifts: {model_findings['total_significant_shifts']}")
            summary_lines.append("")
        
        # Cross-model insights
        if 'cross_model_comparison' in findings:
            comparison = findings['cross_model_comparison']
            common_terms = comparison.get('common_volatile_terms', [])[:5]
            
            if common_terms:
                summary_lines.append("🔄 CROSS-MODEL VALIDATION:")
                summary_lines.append("   Terms consistently volatile across models:")
                for term_data in common_terms:
                    term = term_data['term']
                    volatilities = term_data['volatility_by_model']
                    vol_str = ', '.join(f"{m}:{v:.3f}" for m, v in volatilities.items())
                    summary_lines.append(f"      • {term}: {vol_str}")
                summary_lines.append("")
        
        # Term analysis insights
        term_analysis = findings['term_analysis']
        consistent_terms = list(term_analysis['consistently_volatile_terms'].keys())[:10]
        if consistent_terms:
            summary_lines.append("📈 CONSISTENTLY VOLATILE POLITICAL TERMS:")
            for term in consistent_terms:
                data = term_analysis['consistently_volatile_terms'][term]
                summary_lines.append(f"   • {term}: avg volatility {data['avg_volatility']:.3f}, "
                                    f"trend: {data['dominant_trend']}")
            summary_lines.append("")
        
        # Research implications
        summary_lines.append("💡 RESEARCH IMPLICATIONS:")
        summary_lines.append("   • Political language shows measurable semantic drift over time")
        summary_lines.append("   • Terms like 'reform' and 'security' change ideological associations")
        summary_lines.append("   • Multiple model architectures provide validation of findings")
        summary_lines.append("   • Temporal evaluation crucial for political NLP model robustness")
        summary_lines.append("")
        
        # Write summary
        summary_file = self.output_dir / "RESEARCH_SUMMARY.txt"
        with open(summary_file, 'w') as f:
            f.write('\n'.join(summary_lines))
        
        # Also print to console
        print('\n'.join(summary_lines))
        
        logger.info(f"Created readable summary: {summary_file}")

def main():
    """Main function for political semantic drift analysis"""
    
    # Configuration
    DATA_PATH = "dataset"  # Update path
    OUTPUT_DIR = "report"
    
    print("🔬 Political Semantic Drift Analysis")
    print("Analyzing how political terms shift ideological meaning over time")
    print("=" * 70)
    
    try:
        # Load data
        print("📂 Loading political text data...")
        data_manager = DataManager(DATA_PATH)
        data = data_manager.load_data()
        
        if data.empty:
            raise ValueError("No data loaded")
        
        print(f"✅ Loaded {len(data)} samples across {len(data['period'].unique())} periods")
        print(f"   Periods: {sorted(data['period'].unique())}")
        print(f"   Ideologies: {sorted(data['ideology'].unique())}")
        
        # Period distribution
        period_dist = data.groupby(['period', 'ideology']).size().unstack(fill_value=0)
        print("\n📊 Data distribution by period and ideology:")
        print(period_dist)
        print()
        
        # Initialize experiment
        print("🚀 Initializing semantic drift experiment...")
        experiment = SemanticDriftExperiment(data, output_dir=OUTPUT_DIR)
        
        # Determine available models
        available_models = ["baseline"]
        if TRANSFORMERS_AVAILABLE:
            available_models.extend(["roberta"])  # Add BERT if needed: "bert"
        
        print(f"📋 Available models: {available_models}")
        
        # Run experiment
        print("⚡ Running semantic drift analysis...")
        experiment.run_experiment(
            model_types=available_models,
            min_samples_per_period=50
        )
        
        print("✅ Political semantic drift analysis completed!")
        print(f"📊 Results saved to: {OUTPUT_DIR}")
        print(f"📄 Check RESEARCH_SUMMARY.txt for key findings")
        
    except FileNotFoundError:
        print(f"❌ Data folder not found: {DATA_PATH}")
        print("   Please update DATA_PATH with correct folder location")
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        logger.error(f"Experiment failed: {str(e)}", exc_info=True)

if __name__ == "__main__":
    main()

/Users/I509335/projects/researches/de_proj/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/I509335/projects/researches/de_proj/venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.lau

🔬 Political Semantic Drift Analysis
Analyzing how political terms shift ideological meaning over time
📂 Loading political text data...


INFO:__main__:Loaded DE_CDUParty_2002.csv: 1184 samples
INFO:__main__:Loaded US_RepublicanParty_2004.csv: 1756 samples
INFO:__main__:Loaded US_RepublicanParty_2011.csv: 1521 samples
INFO:__main__:Loaded DE_CDUParty_2017.csv: 1195 samples
INFO:__main__:Loaded DE_AFDParty_2021.csv: 1546 samples
INFO:__main__:Loaded DE_CDUParty_1998.csv: 504 samples
INFO:__main__:Loaded DE_AFDParty_2017.csv: 964 samples
INFO:__main__:Loaded DE_AFDParty_2013.csv: 67 samples
INFO:__main__:Loaded US_DemocraticParty_2000.csv: 1 samples
INFO:__main__:Loaded US_RepublicanParty_2020.csv: 1887 samples
INFO:__main__:Loaded US_RepublicanParty_2008.csv: 983 samples
INFO:__main__:Loaded US_DemocraticParty_2016.csv: 1281 samples
INFO:__main__:Loaded US_DemocraticParty_2012.csv: 1227 samples
INFO:__main__:Loaded US_DemocraticParty_2004.csv: 823 samples
INFO:__main__:Loaded DE_CDUParty_2009.csv: 1711 samples
INFO:__main__:Loaded DE_CDUParty_2021.csv: 2573 samples
INFO:__main__:Total loaded: 26553 samples across 14 years

✅ Loaded 26553 samples across 4 periods
   Periods: ['2000-2005', '2006-2010', '2011-2015', '2016-2022']
   Ideologies: ['LEFT', 'RIGHT']

📊 Data distribution by period and ideology:
ideology   LEFT  RIGHT
period                
2000-2005   824   4135
2006-2010     1   2694
2011-2015  1227   4113
2016-2022  3507  10052

🚀 Initializing semantic drift experiment...
📋 Available models: ['baseline', 'roberta']
⚡ Running semantic drift analysis...


INFO:__main__:Training class distribution: {np.str_('LEFT'): np.int64(824), np.str_('RIGHT'): np.int64(4135)}
INFO:__main__:✅ Successfully trained baseline for 2000-2005
INFO:__main__:Training baseline for period 2006-2010: 2695 samples
INFO:__main__:  Class distribution: {'RIGHT': 2694, 'LEFT': 1}
INFO:__main__:Training baseline model on 2695 samples
INFO:__main__:Training class distribution: {np.str_('LEFT'): np.int64(1), np.str_('RIGHT'): np.int64(2694)}
INFO:__main__:✅ Successfully trained baseline for 2006-2010
INFO:__main__:Training baseline for period 2011-2015: 5340 samples
INFO:__main__:  Class distribution: {'RIGHT': 4113, 'LEFT': 1227}
INFO:__main__:Training baseline model on 5340 samples
INFO:__main__:Training class distribution: {np.str_('LEFT'): np.int64(1227), np.str_('RIGHT'): np.int64(4113)}
INFO:__main__:✅ Successfully trained baseline for 2011-2015
INFO:__main__:Training baseline for period 2016-2022: 13559 samples
INFO:__main__:  Class distribution: {'RIGHT': 10052,

=== POLITICAL SEMANTIC DRIFT ANALYSIS - KEY FINDINGS ===

📊 EXPERIMENT OVERVIEW:
   • Models trained: 4
   • Time periods: 4
   • Model architectures: baseline

🔍 BASELINE MODEL FINDINGS:
   Most semantically volatile terms:
      • green: 0.489 volatility score
      • oil: 0.331 volatility score
      • equality: 0.325 volatility score
      • federal: 0.265 volatility score
      • renewable: 0.256 volatility score
   Terms with ideological direction flips:
      • green: flipped 3 times
      • oversight: flipped 3 times
      • liberty: flipped 2 times
      • insurance: flipped 2 times
      • spending: flipped 2 times
   Trend patterns: {'rightward': 14, 'stable': 5, 'leftward': 11}
   Total significant shifts: 180

💡 RESEARCH IMPLICATIONS:
   • Political language shows measurable semantic drift over time
   • Terms like 'reform' and 'security' change ideological associations
   • Multiple model architectures provide validation of findings
   • Temporal evaluation crucial for po